In [ ]:


import torchaudio
import torch
import numpy as np


import sys
import os

from src.helpers import *

INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [disable_jit_profiling, allow_tf32]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []


In [2]:
import os
print(os.getcwd())

c:\Users\conno\Desktop\UoB_Project\cxb1114


In [ ]:
# load pre-trained model


INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
C:\Users\conno\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\speechbrain\utils\fetching.py:151: UserWarning: Using SYMLINK strategy on Windows for fetching potentially requires elevated privileges and is not recommended. See `LocalStrategy` documentation.
  warnings.warn(
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
C:\Users\conno\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\speechbrain\utils\autocast.py:68: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  wrapped_fwd = torch.cuda.amp.custom_fwd(fwd, cast_inputs=cast_inputs)


In [ ]:
# helper

# def get_embedding(path, model):
#     signal, fs = torchaudio.load(path)
#     with torch.no_grad():
#         embedding = model.encode_batch(signal).squeeze().cpu().numpy()
#     return embedding



# def average_embeddings(file_list):
#     embeddings = [get_embedding(path, model) for path in file_list]
#     return np.mean(embeddings, axis=0)



# def mp3_to_wav(mp3_path, wav_path=None, target_sr=16000):
#     if wav_path is None:
#         wav_path = os.path.splitext(mp3_path)[0] + ".wav"
#     signal, sr = torchaudio.load(mp3_path)
#     if sr != target_sr:
#         resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=target_sr)
#         signal = resampler(signal)
#     torchaudio.save(wav_path, signal, target_sr)
#     return wav_path


In [5]:
# speaker enrollment (listens to audio and creates a large vector containing 'properties' of a voice e.g. pitch, tone, etc.)
# obviously this requires clean single speaker audio


# Aiming to allow for any number of training inputs (theoretically, more clips of a speaker gives better generalisation of their voice attributes)
speaker_list = {'person1': [r'C:\Users\conno\Desktop\UoB_Project\data\commonvoice0104-en\cv-corpus-21.0-delta-2025-03-14\en\clips\common_voice_en_41910499.mp3',
                         r'C:\Users\conno\Desktop\UoB_Project\data\commonvoice0104-en\cv-corpus-21.0-delta-2025-03-14\en\clips\common_voice_en_41910500.mp3',
                         r'C:\Users\conno\Desktop\UoB_Project\data\commonvoice0104-en\cv-corpus-21.0-delta-2025-03-14\en\clips\common_voice_en_41910501.mp3'],
            'person2': [r'C:\Users\conno\Desktop\UoB_Project\data\commonvoice0104-en\cv-corpus-21.0-delta-2025-03-14\en\clips\common_voice_en_41913820.mp3'],
            'person3': [r'C:\Users\conno\Desktop\UoB_Project\data\commonvoice0104-en\cv-corpus-21.0-delta-2025-03-14\en\clips\common_voice_en_41917251.mp3']
            } 

# enrolled_embeddings = {name: average_embeddings(paths) for name, paths in speaker_list.items()}


all_speaker_files = [file for files in speaker_list.values() for file in files]

wav_dir = "./temp"

os.makedirs(wav_dir, exist_ok=True)

wav_files = []

for file in all_speaker_files:
    base = os.path.basename(file)
    wav_path = os.path.join(wav_dir, os.path.splitext(base)[0] + ".wav")
    wav_files.append(mp3_to_wav(file, wav_path))

In [6]:
for i, a in enumerate(wav_files):
    for j, b in enumerate(wav_files):
        if i == 0 and j == 0:
            score = np.zeros((len(wav_files), len(wav_files)))
            prediction = np.zeros((len(wav_files), len(wav_files)))
        score[i, j], prediction[i, j] = model.verify_files(a, b)


In [7]:
prediction

array([[1., 1., 1., 0., 0.],
       [1., 1., 1., 0., 0.],
       [1., 1., 1., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 1.]])

In [8]:
load_audio(wav_files)

Above: ./temp\common_voice_en_41910499.wav



Above: ./temp\common_voice_en_41910500.wav



Above: ./temp\common_voice_en_41910501.wav



Above: ./temp\common_voice_en_41913820.wav



Above: ./temp\common_voice_en_41917251.wav

